In [1]:
%%capture

#enable the process bar
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer
!pip install s3fs
!pip install boto3

print('everything installed')

In [1]:
# %%capture
# !pip install datasets==1.13.3
# !pip install transformers==4.11.3

UsageError: %%capture is a cell magic, but the cell body is empty.


In [1]:
import boto3
import json

In [2]:
from datasets import load_dataset, load_metric, Audio


In [4]:
# import keys from  local 



In [5]:
s3resource = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'] )

In [ ]:
# s3client = boto3.client(
# service_name='s3',
# region_name='eu-west-1',
# aws_access_key_id= data['aws_access_key_id'],
# aws_secret_access_key= data['aws_secret_access_key'] )

In [7]:
BUCKET = 'commonvoicesdataset'
FILE_TO_READ = 'Turkish/vocab.json'
object1 = s3resource.Object(BUCKET,FILE_TO_READ)
file_content = object1.get()['Body'].read().decode('utf-8')
A =json.loads(file_content)

In [8]:
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(A, vocab_file)

In [6]:
from datasets import load_from_disk

In [7]:
from datasets.filesystems import S3FileSystem
s3fsystems = S3FileSystem(key=data['aws_access_key_id'], secret=data['aws_secret_access_key'])  

In [8]:
train_set_loaded = load_from_disk('s3://commonvoicesdataset/Sloveni/train_set',fs=s3fsystems)  

In [9]:
test_set_loaded = load_from_disk('s3://commonvoicesdataset/Sloveni/test_set',fs=s3fsystems)  

In [10]:
test_set_loaded

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 881
})

In [11]:
common_voice_test_transcription = load_dataset("common_voice", "tr", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration tr-ad9f7b76efa9f3a0
Reusing dataset common_voice (/home/ec2-user/.cache/huggingface/datasets/common_voice/tr-ad9f7b76efa9f3a0/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [12]:
!aws s3 cp   s3://commonvoicesdataset/Sloveni/finalmodel  '/home/ec2-user/SageMaker/Sloveniinf' --recursive 

download: s3://commonvoicesdataset/Sloveni/finalmodel/special_tokens_map.json to ../Sloveniinf/special_tokens_map.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/config.json to ../Sloveniinf/config.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/tokenizer_config.json to ../Sloveniinf/tokenizer_config.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/training_args.bin to ../Sloveniinf/training_args.bin
download: s3://commonvoicesdataset/Sloveni/finalmodel/preprocessor_config.json to ../Sloveniinf/preprocessor_config.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/vocab.json to ../Sloveniinf/vocab.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/pytorch_model.bin to ../Sloveniinf/pytorch_model.bin


In [13]:
path_to_model =  '/home/ec2-user/SageMaker/Sloveniinf'

In [14]:
%%capture
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(path_to_model).to("cuda")


In [15]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained(path_to_model)

In [17]:
test_set_loaded[0].keys()

dict_keys(['labels', 'input_values'])

In [16]:
import torch

In [22]:
with open('/home/ec2-user/SageMaker/Sloveniinf/results.text','w') as file:
    

    for sample_num in range(1,200,10):
        input_dict = processor(test_set_loaded[sample_num]["input_values"], return_tensors="pt", padding=True,sampling_rate=16000)

        logits = model(input_dict.input_values.to("cuda")).logits

        pred_ids = torch.argmax(logits, dim=-1)[0]
        
#         print("\nReference: ",common_voice_test_transcription[sample_num]["sentence"].lower())
    #   print(common_voice_test_transcription[sample_num]["sentence"].lower())
#         print("Prediction:",processor.decode(pred_ids))
    #   print(processor.decode(pred_ids))
#         print('------------------------------------------')
        text1 ="\nReference: ",common_voice_test_transcription[sample_num]["sentence"].lower()
        text2 = "\nPrediction:",processor.decode(pred_ids)
#         print('------------------------------------------')
#         file.write(text1)
#         file.write(text2)
        file.writelines(text1)
        file.writelines(text2)
        file.writelines('\n---------------------------------')

/home/ec2-user/anaconda3/envs/s2t/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811756765/work/torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
